# Gradient Calculation

## 1. Introduction

The gradient function uses the Discrete Adjoint with secord-order in time fully-explicit timestepping scheme with implementation of a Perfectly Matched Layer (PML) using CG FEM with or without higher order mass lumping (KMV type elements).

The gradient calculation algorithm follows the steps:
1. a synthetic FWI problem is runned in order to generate a synthetic *shot record* for reference
2. another FWI problem is executed, but with the initial guess
3. the difference between the results is evaluated as follows
$$
misfit = p_{exact_{recv}} - p_{guess_{recv}}
$$
4. then, the misfit and other parameters, such as the mesh, the velocity model guess and the receivers location, are used as input to the `gradient` function

The gradient calculation method is based on the **adjoint method**, because it reduces the computational cost in a problem with many variables. Besides that, the adjoint equations have as source the difference in the receivers between the real values and the guess.

## 2. Example

### 2.1. Real case

The first step is to evaluate the real case. It includes the libraries,

In [9]:
from firedrake import File
import spyro
import numpy as np

model definition,

In [4]:
model = {}
model["opts"] = {
    "method": "KMV",  # either CG or KMV
    "quadratrue": "KMV",  # Equi or KMV
    "degree": 4,  # p order
    "dimension": 2,  # dimension
}
model["parallelism"] = {
    "type": "automatic",
}
model["mesh"] = {
    "Lz": 3.5,  # depth in km - always positive
    "Lx": 17.0,  # width in km - always positive
    "Ly": 0.0,  # thickness in km - always positive
    "meshfile": "meshes/marmousi_exact.msh",
    "initmodel": "velocity_models/not_used.hdf5",
    "truemodel": "velocity_models/marmousi_exact.hdf5",
}
model["BCs"] = {
    "status": True,  # True or false
    "outer_bc": "non-reflective",  #  None or non-reflective (outer boundary condition)
    "damping_type": "polynomial",  # polynomial, hyperbolic, shifted_hyperbolic
    "exponent": 2,  # damping layer has a exponent variation
    "cmax": 4.5,  # maximum acoustic wave velocity in PML - km/s
    "R": 1e-6,  # theoretical reflection coefficient
    "lz": 0.9,  # thickness of the PML in the z-direction (km) - always positive
    "lx": 0.9,  # thickness of the PML in the x-direction (km) - always positive
    "ly": 0.0,  # thickness of the PML in the y-direction (km) - always positive
}
model["acquisition"] = {
    "source_type": "Ricker",
    "num_sources": 40,
    "source_pos": spyro.create_transect((-0.01, 1.0), (-0.01, 15.0), 40),
    "frequency": 5.0,
    "delay": 1.0,
    "num_receivers": 500,
    "receiver_locations": spyro.create_transect((-0.10, 0.1), (-0.10, 17.0), 500),
}
model["timeaxis"] = {
    "t0": 0.0,  #  Initial time for event
    "tf": 5.00,  # Final time for event
    "dt": 0.00025,
    "amplitude": 1,  # the Ricker has an amplitude of 1.
    "nspool": 100,  # how frequently to output solution to pvds
    "fspool": 99999,  # how frequently to save solution to RAM
}

and model evaluation

In [5]:
comm = spyro.utils.mpi_init(model)
mesh, V = spyro.io.read_mesh(model, comm)
vp = spyro.io.interpolate(model, mesh, V, guess=False)
if comm.ensemble_comm.rank == 0:
    File("true_velocity.pvd", comm=comm.comm).write(vp)
sources = spyro.Sources(model, mesh, V, comm)
receivers = spyro.Receivers(model, mesh, V, comm)
wavelet = spyro.full_ricker_wavelet(
    dt=model["timeaxis"]["dt"],
    tf=model["timeaxis"]["tf"],
    freq=model["acquisition"]["frequency"],
)
p, p_r = spyro.solvers.forward(model, mesh, comm, vp, sources, wavelet, receivers)
spyro.io.save_shots(model, comm, p_r)

1


ValueError: Available cores cannot be divided between sources equally.

### 2.2. Guess model

Similarly, the guess model is submitted to the same process, except for the `save_shots` function.

In [6]:
model = {}

model["opts"] = {
    "method": "KMV",  # either CG or KMV
    "quadratrue": "KMV",  # Equi or KMV
    "degree": 5,  # p order
    "dimension": 2,  # dimension
}
model["parallelism"] = {
    "type": "automatic",
}
model["mesh"] = {
    "Lz": 3.5,  # depth in km - always positive
    "Lx": 17.0,  # width in km - always positive
    "Ly": 0.0,  # thickness in km - always positive
    "meshfile": "meshes/marmousi_guess.msh",
    "initmodel": "velocity_models/marmousi_guess.hdf5",
    "truemodel": None,
}
model["BCs"] = {
    "status": True,  # True or false
    "outer_bc": "non-reflective",  #  None or non-reflective (outer boundary condition)
    "damping_type": "polynomial",  # polynomial, hyperbolic, shifted_hyperbolic
    "exponent": 2,  # damping layer has a exponent variation
    "cmax": 4.5,  # maximum acoustic wave velocity in PML - km/s
    "R": 1e-6,  # theoretical reflection coefficient
    "lz": 0.9,  # thickness of the PML in the z-direction (km) - always positive
    "lx": 0.9,  # thickness of the PML in the x-direction (km) - always positive
    "ly": 0.0,  # thickness of the PML in the y-direction (km) - always positive
}
model["acquisition"] = {
    "source_type": "Ricker",
    "num_sources": 40,
    "source_pos": spyro.create_transect((-0.01, 1.0), (-0.01, 15.0), 40),
    "frequency": 5.0,
    "delay": 1.0,
    "num_receivers": 500,
    "receiver_locations": spyro.create_transect((-0.10, 0.1), (-0.10, 17.0), 500),
}
model["timeaxis"] = {
    "t0": 0.0,  #  Initial time for event
    "tf": 5.00,  # Final time for event
    "dt": 0.001,
    "amplitude": 1,  # the Ricker has an amplitude of 1.
    "nspool": 100,  # how frequently to output solution to pvds
    "fspool": 99999,  # how frequently to save solution to RAM
}

In [7]:
comm = spyro.utils.mpi_init(model)
mesh, V = spyro.io.read_mesh(model, comm)
vp = spyro.io.interpolate(model, mesh, V, guess=True)
if comm.ensemble_comm.rank == 0:
    File("true_velocity.pvd", comm=comm.comm).write(vp)
sources = spyro.Sources(model, mesh, V, comm)
receivers = spyro.Receivers(model, mesh, V, comm)
wavelet = spyro.full_ricker_wavelet(
    dt=model["timeaxis"]["dt"],
    tf=model["timeaxis"]["tf"],
    freq=model["acquisition"]["frequency"],
)
p, p_r = spyro.solvers.forward(model, mesh, comm, vp, sources, wavelet, receivers)

1


ValueError: Available cores cannot be divided between sources equally.

If there is water inside the domain, it should be localized:

In [10]:
water = np.where(vp.dat.data[:] < 1.51)

NameError: name 'vp' is not defined

### 2.3. Gradient calculation

The gradient is calculated according the adjoint method. The receivers are acting as sources and the misfit is the recorded value of the sources:

In [12]:
dJ = Function(V, name="gradient")
dJ_local = spyro.solvers.gradient(
    model,
    mesh,
    comm,
    vp,
    receivers,
    self.p_guess,
    self.misfit,
)
if comm.ensemble_comm.size > 1:
    comm.allreduce(dJ_local, dJ)
else:
    dJ = dJ_local
dJ /= comm.ensemble_comm.size
if comm.comm.size > 1:
    dJ /= comm.comm.size
# regularize the gradient if asked.
if model["opts"]["regularization"]:
    dJ = regularize_gradient(vp, dJ)
# mask the water layer
dJ.dat.data[water] = 0.0

NameError: name 'V' is not defined